In [1]:
from requests_html import HTMLSession
import wikipedia
import warnings
import spacy
import geonamescache
import csv
import pandas as pd

In [2]:
gc = geonamescache.GeonamesCache()

# gets nested dictionary for countries
countries = gc.get_countries()

# gets nested dictionary for cities
cities = gc.get_cities()

In [3]:
def gen_dict_extract(var, key):
    if isinstance(var, dict):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, (dict, list)):
                yield from gen_dict_extract(v, key)
    elif isinstance(var, list):
        for d in var:
            yield from gen_dict_extract(d, key)

In [4]:
cities = [*gen_dict_extract(cities, 'name')]

In [7]:
def get_url(string):
    url = ''
    try:
        # get page url from wikipedia
        page = wikipedia.page(string)
        url = page.url
    except wikipedia.exceptions.PageError:
        #too see weather there is one this page
        pass
    except wikipedia.exceptions.DisambiguationError as e:
        get_url(e.options[0])
    return url

In [8]:
def get_city_list(string): 
  warnings.filterwarnings('ignore')
  nlp = spacy.load('en')
  doc = nlp(string)
  sentences = list(doc.sents)
  ls = []
  # get the location words
  for sentence in sentences:
    process = nlp(sentence.text).ents
    for token in process:
      #print(token)
      if token.text in cities:
        ls.append(token.text)
  return ls

In [18]:
def get_webinfo(url):
    result = ''
    if len(url) == 0:
        return result
    session = HTMLSession()
    r = session.get(url)
    sel1='#mw-content-text > div.mw-parser-output > table'
    sel2='#mw-content-text > div.mw-parser-output > table.infobox.vcard'
    ls_sel=[sel1,sel2]
    for i in ls_sel:
        result1 = r.html.find(i)
        if len(result1) == 0:
            continue
        result = result1[0].text.replace("\n", " ")
        #print(result)
        if result[:18]!='This article needs':
            break
    #print(result)
    return result

In [10]:
def get_related_city(search,ls):
  dictionary = {}
  for key in ls:
      dictionary[key] = dictionary.get(key, 0) + 1
  tem = sorted(dictionary.items(), key=lambda d: d[1])
  tem_list = list(tem)
  #return tem_list
  for i in tem_list:
      #print(tem_list[-1][0])
      if tem_list[-1][0] not in search:
          return tem_list[-1][0]
      else:
          tem_list = tem_list[:-1]

In [37]:
def get_city(search):
    url = get_url(search)
    if url =='':
        return None
    tem = get_webinfo(url)
    if tem =='':
        return None
    ls = get_city_list(tem)
    if ls ==[]:
        return None
    result = get_related_city(search,ls)
    return result

In [51]:
get_city("Donald Trump")

'New York City'

In [12]:
word_ls = []
city_ls = []

In [17]:
#get the trending words of Twitter
file = open("./demo_trending_word.txt")
#save the trending words and its related city to list
lines = file.readlines()
for line in lines:
    #print(line)
    tem = get_city(line.strip())
    if tem != None:
        word_ls.append(line)
        city_ls.append(tem)

In [20]:
#save the trending words and its related city to csv file
dataframe = pd.DataFrame({'trending word':word_ls,'related city':city_ls})

dataframe.to_csv("demo_related_city.csv",index=False)